<a href="https://colab.research.google.com/github/hamidzangiabadi/LSTM-HHO-IoMT-Colab/blob/main/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hello")

hello
